In [1]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
driver = webdriver.Chrome("C:\\Users\\Dell\\.wdm\\drivers\\chromedriver.exe")

In [2]:
url="https://www.amazon.in"
driver.get(url)

In [3]:
def get_url(search_term):
    template = 'https://www.amazon.in/s?k=smartphones&rh=n%3A1389401031&ref=nb_sb_noss'
    search_term = search_term.replace('', '+')
    return template.format(search_term)
url = get_url('smartphones')
driver.get(url)

In [4]:
soup = BeautifulSoup(driver.page_source,'html.parser')
results = soup.find_all('div',{'data-component-type': 's-search-result'})
item = results[0]

In [5]:
def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.in' + atag.get('href')
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    result = (description, price, rating, review_count, url)
    return result

    

In [6]:
records = []
results = soup.find_all('div',{'data-component-type': 's-search-result'})

In [7]:
for item in results:
    record = extract_record(item)
    if record:
        records.append(record)


In [8]:
def get_url_format(search_term):
    template = 'https://www.amazon.in/s?k=smartphones&rh=n%3A1389401031&ref=nb_sb_noss'
    search_term = search_term.replace('', '+')
    url = template.format(search_term)
    url += '&page()'
    return url

In [9]:
def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.in' + atag.get('href')
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
            return
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    result = (description, price, rating, review_count,url)
    return result

In [10]:
def main(search_term):
    record = []
    url = get_url(search_term)
    for page in range(1,21):
        get_url_format(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'data-component-type': 's-search-result'})
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
       
        with open('results.csv','w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow({'Description', 'Price', 'Rating', 'Reviewcount','Url'})
            writer.writerows(records)

In [11]:
main('smartphones')
driver.close()